In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits
import os
import sys
from shutil import copyfile

In [5]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [6]:
cat = Table.read('./Catalog/NSA/z002_004/UDG_NSA_MW_z002_004_JG.fits')

In [7]:
len(cat)

105

In [20]:
cat['viz-id'].data

array([  931,  1760,  2071,  4032,  4188,  4221,  4363,  5051,  6675,
       11959, 12500, 12814, 13292, 14037, 14970, 15381, 15401, 17439,
       17947, 18077, 19656, 19965, 21244, 21384, 21531, 22086, 23739,
       24267, 25205, 27799, 28347, 29725, 30027, 30675, 32512, 33007,
       33394, 34727, 35129, 38086, 38470, 39390, 40671, 40955, 41290,
       41501, 44405, 44841, 44926, 45970, 46914, 47949, 48838, 49557,
       51149, 51341, 51815, 52367, 52915, 54980, 57828, 58901, 59939,
       60199, 60461, 60721, 62121, 62214, 62470, 63264, 63430, 64077,
       64379, 64482, 64638, 65334, 66772, 69656, 70143, 72306, 74461,
       74926, 75879, 76145, 78228, 79056, 79133, 79706, 80065, 81294,
       81343, 81848, 82230, 82983, 83554, 84101, 84314, 85545, 85715,
       86966, 88425, 88587, 89144, 89269, 90593])

In [8]:
category = "UDG_new"

### Generate RGB images

In [9]:
from kuaizi.display import display_HSC_cutout_rgb

In [10]:
CUTOUT_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/'
FIGURE_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/'
WEBPAGE_FIG_DIR = f'/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/figure/{category}/'

In [11]:
problem_ids = []
# Check file exsistence
for filt in 'grizy':
    for obj in cat:
        if not os.path.isfile(CUTOUT_DIR + f"nsa_{obj['viz-id']}_{filt}.fits"):
            #print(f"NSA/nsa_{obj['viz-id']}_{filt}.fits not found!")
            problem_ids.append(obj['viz-id'])
            
if not os.path.isdir(FIGURE_DIR):
    os.mkdir(FIGURE_DIR)

In [12]:
print('# of no cutouts:', np.unique(problem_ids).shape)

# of no cutouts: (0,)


In [13]:
# Display each cutout
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try:
        images = np.asarray([fits.open(f'{CUTOUT_DIR}/nsa_{ind}_{filt}.fits')[1].data for filt in 'griz'])
        fig, ax = plt.subplots(figsize=(5, 5))
        size = int(obj['rhalf_ellip'] * 5) if obj['rhalf_ellip'] * 5 > 100 else 100
        ax = display_HSC_cutout_rgb(images, ax=ax, half_width=size)
        plt.savefig(FIGURE_DIR + f'nsa_{ind}_cutout.png', dpi=70, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(ind, e)
        
    if i % 10 == 0:
        print(f'Progress: {i} / {len(cat)}')

Progress: 0 / 105
Progress: 10 / 105
Progress: 20 / 105
Progress: 30 / 105
Progress: 40 / 105
Progress: 50 / 105
Progress: 60 / 105
Progress: 70 / 105
Progress: 80 / 105
Progress: 90 / 105
Progress: 100 / 105


In [21]:
# Copy figures in FIGURE_DIR to WEBPAGE_FIG_DIR
if not os.path.isdir(WEBPAGE_FIG_DIR):
    os.mkdir(WEBPAGE_FIG_DIR)
else:
    import glob
    fileList = glob.glob(WEBPAGE_FIG_DIR + '*.png', recursive=True)
    [os.remove(file) for file in fileList];
            
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try: 
        copyfile(FIGURE_DIR + f'nsa_{ind}_cutout.png', WEBPAGE_FIG_DIR + f'nsa_{ind}_cutout.png')
    except Exception as e:
        print(ind, e)

### Generate webpage for cutouts. We use `diezi/webpage/gen_web_cutout.py`

In [26]:
sys.path.append('/home/jiaxuanl/Research/Packages/kuaizi/diezi/webpage/')
from gen_web_cutout import webpage_cutout

In [34]:
#FOR NSA SAMPLE
WEBPAGE_FIG_DIR = f'/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/figure/{category}/'
WEBPAGE_DIR = f'/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/page/{category}/'
SCARLET_DIR = None #'/tigress/jiaxuanl/public_html/NSA/scarlet_result/'
sample_name = 'NSA'

In [35]:
# Non-UDG based on visual inspection
bad_idx = np.array([1760, 12500, 17439, 28347, 
                30027, 30675, 39390, 41501, 49557, 
                63848, 64077, 64379, 66143, 
                78228, 81848, 86966, ])

In [38]:
webpage_cutout(WEBPAGE_FIG_DIR, WEBPAGE_DIR, sample_name, sample_title=f'{category}', SCARLET_DIR=SCARLET_DIR, 
               col_num=6, row_num=4, colored_idx=bad_idx, color='red')

You have 105 galaxies to be displayed
Total pages: 5


In [39]:
print('The webpages are in', WEBPAGE_DIR)

The webpages are in /tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/page/UDG_new/


### The search box on webpage still doesn't work. Need to work on the javascript file